#  <u> 1st step : Load

Open the final merge table of the 'integration part '

In [8]:
# we import the useful librairies 

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import functools
import os
import plotly.express as px
from scipy.stats import shapiro
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual


# we open and read the merge table of all indicators
bronze_dataset = pd.read_csv ('bronze_dataset.csv')
bronze_dataset = bronze_dataset.drop(bronze_dataset.columns[0], axis = 1)
bronze_dataset

,Code,Year,Indicator,Value
0,AFG,2005,Deaths,1.186530e+05
1,AFG,2005,LifeExpectancy,5.840000e+01
2,AFG,2005,GenderInequality,7.480000e-01
3,AFG,2005,GDP,3.451279e+09
4,AFG,2005,Fertility,6.904700e+00
...,...,...,...,...
3517,ZWE,2010,LifeExpectancy,5.070000e+01
3518,ZWE,2010,GenderInequality,5.920000e-01
3519,ZWE,2010,GDP,6.097825e+09
3520,ZWE,2010,Fertility,4.024800e+00


# <u>STEP 2 : Normalization  

We compute the quantiles :  first quartile (Q1=25%) and the third quartile (Q3=75%). Then the interquartile range.

for each country and each indicators 


In [9]:
# 1st quartile 
Q1=bronze_dataset.groupby(['Code','Indicator']).quantile(0.25)

# 3rd quartile 
Q3=bronze_dataset.groupby(['Code','Indicator']).quantile(0.75)

#interquartile range 
IQR=Q3-Q1

IQR

Year         Value
Code Indicator                             
AFG  Deaths               2.5  8.497500e+03
     Fertility            2.5  4.030000e-01
     GDP                  2.5  3.431342e+09
     GenderInequality     2.5  2.500000e-03
     LifeExpectancy       2.5  1.250000e+00
...                       ...           ...
ZWE  Fertility           10.0  1.799000e-01
     GDP                 10.0  5.252434e+08
     GenderInequality    10.0  1.100000e-02
     LifeExpectancy      10.0  5.900000e+00
     tertiary_education  10.0  8.800000e-01

[756 rows x 2 columns]

Then we compute the upper and lower limit, and we delete the column year and rename the Value column

In [10]:
lower_limit =Q1 - 1.5 * IQR
lower_table =lower_limit.drop(['Year'],axis=1)
lower_table.rename(columns={"Value":"Lower limit"})

Lower limit
Code Indicator                       
AFG  Deaths              9.316050e+04
     Fertility           5.695700e+00
     GDP                 1.993741e+07
     GenderInequality    7.455000e-01
     LifeExpectancy      5.715000e+01
...                               ...
ZWE  Fertility           3.704150e+00
     GDP                 3.633735e+09
     GenderInequality    5.775000e-01
     LifeExpectancy      3.595000e+01
     tertiary_education -8.800000e-01

[756 rows x 1 columns]

In [11]:
upper_limit=Q3 + 1.5 * IQR
upper_table=upper_limit.drop(['Year'],axis=1)
upper_table.rename(columns={"Value":"Upper limit"})

Upper limit
Code Indicator                       
AFG  Deaths              1.271505e+05
     Fertility           7.307700e+00
     GDP                 1.374530e+10
     GenderInequality    7.555000e-01
     LifeExpectancy      6.215000e+01
...                               ...
ZWE  Fertility           4.423750e+00
     GDP                 5.734709e+09
     GenderInequality    6.215000e-01
     LifeExpectancy      5.955000e+01
     tertiary_education  2.640000e+00

[756 rows x 1 columns]


Then we merge the three tables : the Bronze_dataset, the upper_table and the lower_table
However, we use the functions reduce from functools
It allows to merge the three tables in one command

In [12]:
three_tables = [bronze_dataset,lower_table,upper_table]
tables_joined = functools.reduce(lambda left, right: pd.merge(left, right, on=['Code','Indicator']), three_tables)
tables_joined

,Code,Year,Indicator,Value_x,Value_y,Value
0,AFG,2005,Deaths,118653.000,93160.5000,127150.5000
1,AFG,2010,Deaths,101658.000,93160.5000,127150.5000
2,AFG,2005,LifeExpectancy,58.400,57.1500,62.1500
3,AFG,2010,LifeExpectancy,60.900,57.1500,62.1500
4,AFG,2005,GenderInequality,0.748,0.7455,0.7555
...,...,...,...,...,...,...
3517,ZWE,1990,tertiary_education,1.830,-0.8800,2.6400
3518,ZWE,1995,tertiary_education,1.320,-0.8800,2.6400
3519,ZWE,2000,tertiary_education,0.530,-0.8800,2.6400
3520,ZWE,2005,tertiary_education,0.440,-0.8800,2.6400


We rename the columns in order to understand better

In [13]:
renamed=tables_joined.set_axis(['Code','Year','Indicator', 'Real value', 'Lower value', 'Upper value'], axis=1)
renamed

,Code,Year,Indicator,Real value,Lower value,Upper value
0,AFG,2005,Deaths,118653.000,93160.5000,127150.5000
1,AFG,2010,Deaths,101658.000,93160.5000,127150.5000
2,AFG,2005,LifeExpectancy,58.400,57.1500,62.1500
3,AFG,2010,LifeExpectancy,60.900,57.1500,62.1500
4,AFG,2005,GenderInequality,0.748,0.7455,0.7555
...,...,...,...,...,...,...
3517,ZWE,1990,tertiary_education,1.830,-0.8800,2.6400
3518,ZWE,1995,tertiary_education,1.320,-0.8800,2.6400
3519,ZWE,2000,tertiary_education,0.530,-0.8800,2.6400
3520,ZWE,2005,tertiary_education,0.440,-0.8800,2.6400


In [14]:
renamed.to_csv('bronze_dataset_with_outliers.csv')